In [1]:
import pandas as pd 
import numpy as np  
import re
import random
import json 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

path = 'C:/Users/GMEU0170/OneDrive - Hyundai Motor Europe/GME_CD/Documents/2.__Code/_.py/web_scraping/selenium/'

In [2]:
# Set up Chrome options (headless mode is optional)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
# Use webdriver-manager to install and manage ChromeDriver
service = Service(ChromeDriverManager().install())

In [4]:
# User-Agent list for rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    # Add more user agents as needed
]

In [5]:
def get_driver(service):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    ua = random.choice(user_agents)
    chrome_options.add_argument(f"user-agent={ua}")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    # Explicitly set the page load strategy to "normal"
    chrome_options.page_load_strategy = "normal"
    
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.implicitly_wait(10)
    return driver

In [6]:
driver = get_driver(service)

### Testing

In [7]:
url = "https://configure.bmw.de/de_DE/configure/U10/51GM/FBLAT,P0C68,S01EE,S0230,S02PA,S02VB,S02VD,S0302,S0428,S0478,S0494,S04GN,S04T2,S04U9,S04V1,S0548,S05AV,S05DM,S0654,S06AE,S06AF,S06PA,S0760,S0801,S0851,S0879,S08R3,S08R9,S08TF/SE000001"

In [8]:
driver.get(url)
time.sleep(10)

In [9]:
wait = WebDriverWait(driver, 15)
shadow_host = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "con-app")))

In [10]:
shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)

In [11]:
def expand_shadow_element(driver, element):
    return driver.execute_script("return arguments[0].shadowRoot", element)

# Example usage:
con_app = driver.find_element(By.CSS_SELECTOR, "con-app")
shadow1 = expand_shadow_element(driver, con_app)

router_slot = shadow1.find_element(By.CSS_SELECTOR, "router-slot")
shadow2 = expand_shadow_element(driver, router_slot)

In [12]:
slot = shadow2.find_element(By.CSS_SELECTOR, "slot")
slot

assigned_nodes = driver.execute_script("""
    let slot = arguments[0];
    return slot.assignedNodes();
""", slot)

# Print the assigned nodes to verify we have the correct elements
for node in assigned_nodes:
    print(node.tag_name)
    
con_resource_provider_page = None
for node in assigned_nodes:
    if node.tag_name == "con-resource-provider-page":
        con_resource_provider_page = node
        break
    
if con_resource_provider_page:
    print("Found con-resource-provider-page")
    # You can perform further actions with con_resource_provider_page
else:
    print("con-resource-provider-page not found")

con-resource-provider-page
Found con-resource-provider-page


In [13]:
# Get all the children of the con_resource_provider_page element
children = con_resource_provider_page.find_elements(By.XPATH, "./*")

# Print the tag names of the children
for child in children:
    print(child.tag_name)

con-configure


In [14]:
# Find the 'con-configure' element inside 'con-resource-provider-page'
con_configure = con_resource_provider_page.find_element(By.CSS_SELECTOR, "con-configure")

con_configure_shadow_root = expand_shadow_element(driver, con_configure)

In [18]:
children_under_shadow = driver.execute_script("""
    const conConfigure = arguments[0];
    const shadowRoot = conConfigure.shadowRoot;
    return Array.from(shadowRoot.children).map(child => child.tagName);
""", con_configure)

for child in children_under_shadow:
    print(child)

COSY-DYNAMIC-NAMED-IMAGES
COSY-DYNAMIC-NAMED-IMAGES
COSY-DYNAMIC-NAMED-IMAGES
DIV
CON-SALES-BAR


In [40]:
con_sales_bar = driver.execute_script("""
    const conConfigure = arguments[0];
    return conConfigure.shadowRoot.querySelector("con-sales-bar");
""", con_configure)

children_under_con_sales_bar = driver.execute_script("""
    const conConfigure = arguments[0];
    const shadowRoot = conConfigure.shadowRoot;
    return Array.from(shadowRoot.children).map(child => child.tagName);
""", con_sales_bar)

for child in children_under_con_sales_bar:
    print(child)

CON-LINK-PROXY
DIV


In [50]:
con_sales_bar_shadow_root = driver.execute_script("""
    return arguments[0].shadowRoot;
""", con_sales_bar)

# Step 2: Get the <div> child inside the shadow root
div_inside_con_sales_bar = driver.execute_script("""
    return arguments[0].shadowRoot.querySelector("div");
""", con_sales_bar)

div_inside_con_sales_bar.get_attribute("outerHTML")

'<div class="compact-sales-bar-wrapper"><div class="compact-info-elements-wrapper"><div class="price-wrapper"><!--?lit$1165044204$--><div class="total-price  price-border "><!--?lit$1165044204$--><!--?lit$1165044204$--><a data-name="price" class="link compact" role="button" title="Preisübersicht"><span class="price"><!--?lit$1165044204$-->53.250&nbsp;€</span></a> <!--?lit$1165044204$--><!--?--></div> <span><!--?lit$1165044204$--></span></div><!--?lit$1165044204$--><button class="button link" data-key="SALESBAR_OVERLAY" aria-label="Mehr Information"><span><!--?lit$1165044204$-->Mehr Information</span> <i class="button-icon con-icon icon-external-link" aria-hidden="true"></i></button></div><div class="compact-ctas-wrapper"><!--?lit$1165044204$--><!--?lit$1165044204$--><!----><button class="button button-primary button-square" data-modal-trigger="" data-key="SAVE" tooltip="" title="Konfiguration speichern"><!--?lit$1165044204$--><i aria-hidden="true" class="button-icon con-icon icon-save"

In [ ]:
# <div class="sales-bar"?
#     <div class="model-name-section">
#     <div class="sales-footer-sections">
#         <div class="sales-footer-sections-top">
#             <div class="sales-footer-prices-section">
#                 <div tabindex="0" class="finance-container  ">
#                     <!--?lit$898154033#-->
#                     <sf-cf-integration-widget app='CONX' type="variant-installment" page="Configurator">
#                         #shadow-root (open)

<selenium.webdriver.remote.webelement.WebElement (session="b752ee1d62b23777e7baf1adde8597c5", element="f.7AE0CFE899815E699D4B93EC1A5C5679.d.1B357EA3F59E35611FBB0118FD0A7524.e.25")>

In [66]:
children_under_div = driver.execute_script("""
    const div = arguments[0];
    const children = Array.from(div.children);
    return children.map(child => child.tagName);  // Returns a list of tag names of all children
""", div_inside_con_sales_bar)

# Print the list of children tag names
for html in children_under_div:
    print(html)

DIV
DIV


In [72]:
second_div = driver.execute_script("""
    return arguments[0].children[1];
""", div_inside_con_sales_bar)

sales_footer = driver.execute_script("""
    return arguments[0].querySelector('div div div.sales-footer-sections');
""", second_div)

sales_footer

In [73]:
all_inner_html = driver.execute_script("""
    return arguments[0].innerHTML;
""", second_div)

print(all_inner_html)

<!--?lit$1165044204$--><!--?lit$1165044204$--><!----><button class="button button-primary button-square" data-modal-trigger="" data-key="SAVE" tooltip="" title="Konfiguration speichern"><!--?lit$1165044204$--><i aria-hidden="true" class="button-icon con-icon icon-save"></i></button><!----> <button class="button button-tertiary button-square   " data-key="SALESBAR_OVERLAY" aria-label=""><i aria-hidden="true" class="button-icon con-icon icon-chevron-up"></i></button>


In [75]:
elements = driver.execute_script("""
    function getElementTree(element) {
        let result = [];
        function walk(el, depth) {
            result.push('  '.repeat(depth) + el.tagName + (el.className ? ' [' + el.className + ']' : ''));
            Array.from(el.children).forEach(child => walk(child, depth + 1));
        }
        walk(element, 0);
        return result;
    }
    return getElementTree(arguments[0]);
""", second_div)

for line in elements:
    print(line)


DIV [compact-ctas-wrapper]
  BUTTON [button button-primary button-square]
    I [button-icon con-icon icon-save]
  BUTTON [button button-tertiary button-square   ]
    I [button-icon con-icon icon-chevron-up]


In [58]:
second_div = driver.execute_script("""
    const div = arguments[0];
    const children = Array.from(div.children);
    return children[1];  // Access the second DIV element
""", div_inside_con_sales_bar)

# Print the outer HTML of the second DIV to verify
print(second_div.get_attribute("outerHTML"))


<div class="compact-ctas-wrapper"><!--?lit$1165044204$--><!--?lit$1165044204$--><!----><button class="button button-primary button-square" data-modal-trigger="" data-key="SAVE" tooltip="" title="Konfiguration speichern"><!--?lit$1165044204$--><i aria-hidden="true" class="button-icon con-icon icon-save"></i></button><!----> <button class="button button-tertiary button-square   " data-key="SALESBAR_OVERLAY" aria-label=""><i aria-hidden="true" class="button-icon con-icon icon-chevron-up"></i></button></div>


In [61]:
children_of_second_div = driver.execute_script("""
    const div = arguments[0];
    const children = Array.from(div.children);
    return children.map(child => child.outerHTML);  // Get outerHTML of all children
""", second_div)

# Print all children
for child_html in children_of_second_div:
    print(child_html)

<button class="button button-primary button-square" data-modal-trigger="" data-key="SAVE" tooltip="" title="Konfiguration speichern"><!--?lit$1165044204$--><i aria-hidden="true" class="button-icon con-icon icon-save"></i></button>
<button class="button button-tertiary button-square   " data-key="SALESBAR_OVERLAY" aria-label=""><i aria-hidden="true" class="button-icon con-icon icon-chevron-up"></i></button>


In [62]:
children_elements = driver.execute_script("""
    const parent = arguments[0];
    return Array.from(parent.querySelectorAll(':scope > *')).map(el => el.outerHTML);
""", second_div)

for html in children_elements:
    print(html)

<button class="button button-primary button-square" data-modal-trigger="" data-key="SAVE" tooltip="" title="Konfiguration speichern"><!--?lit$1165044204$--><i aria-hidden="true" class="button-icon con-icon icon-save"></i></button>
<button class="button button-tertiary button-square   " data-key="SALESBAR_OVERLAY" aria-label=""><i aria-hidden="true" class="button-icon con-icon icon-chevron-up"></i></button>


<i aria-hidden="true" class="button-icon con-icon icon-save"></i>
<i aria-hidden="true" class="button-icon con-icon icon-chevron-up"></i>
